In [1]:
import numpy as np
import pandas as pd
import os
import math
from operator import truediv
import os
import random
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.nn.modules.loss import _Loss
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Global Aşama

In [3]:
seed = 42

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(seed)


windowSize = 35
batch_size = 64


def padding(X, margin=2):
    newX = np.zeros((X.shape[0] + 2*margin, X.shape[1] + 2*margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset : X.shape[0] + x_offset, y_offset : X.shape[1] + y_offset, :] = X
    return newX

def createPatch(X, y, windowSize=windowSize):
    margin = int((windowSize-1)/2)
    zeroPaddedX = padding(X, margin=margin)

    patchesData = np.zeros((X.shape[0]*X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0


    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin: r + margin + 1, c - margin:c + margin + 1]
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex += 1

    patchesData = patchesData[patchesLabels > 0, :, :, :]
    patchesLabels = patchesLabels[patchesLabels > 0]
    patchesLabels -= 1


    patchesData = patchesData.transpose((0, 3, 1, 2))

    return patchesData, patchesLabels


def applyPCA(X, numComponents=15):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX


def splitTrainTestSet(X, y, randomState=42):
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.7, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.40, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test



**VeriSeti**


In [4]:
X=np.load("/content/drive/MyDrive/dml/indian-veriseti/indianpinearray.npy")
y=np.load("/content/drive/MyDrive/dml/indian-veriseti/IPgt.npy")

X=applyPCA(X)

X,y=createPatch(X,y)

print(X.shape)
print(y.shape)
device = torch.device("cuda")


X_train,X_val,X_test,y_train,y_val,y_test=splitTrainTestSet(X,y)

(10249, 15, 35, 35)
(10249,)


In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [12]:
class HyperSpectralDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]

        if self.transform:
            sample = self.transform(sample)

        return sample, label



train_dataset = HyperSpectralDataset(
    data=torch.tensor(X_train, dtype=torch.float32),
    labels=torch.tensor(y_train, dtype=torch.long)
    )

val_dataset = HyperSpectralDataset(
    data=torch.tensor(X_val, dtype=torch.float32),
    labels=torch.tensor(y_val, dtype=torch.long)
    )


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=64,shuffle=True)

# Modeller

In [5]:
import torch.nn as nn
import torchvision.models as models


**Resnet**

In [ ]:

# ####  Resnet #########

# model = models.resnet18(pretrained=True)
# model.conv1 = nn.Conv2d(15, 64, kernel_size=7, stride=2, padding=3, bias=False)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 16)

# model.to(device)





/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 143MB/s]


**DenseNet**

In [ ]:
######### DenseNet  #########

model = models.densenet121(pretrained=True)
original_conv1 = model.features.conv0
model.features.conv0 = nn.Conv2d(15, original_conv1.out_channels, kernel_size=original_conv1.kernel_size,
                                 stride=original_conv1.stride, padding=original_conv1.padding, bias=False)
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 16)


model.to(device)

NameError: name 'models' is not defined

**İnceptionNet**

In [ ]:
model = models.googlenet(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 16)
model.to(device)


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

**Efficient Net**

In [ ]:
model = models.efficientnet_b0(pretrained=True)
model.features[0][0] = nn.Conv2d(15, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
model.to(device)


Görselleştirme

In [ ]:

num_epochs = 20
learning_rate = 0.001
num_classes = 16

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
for epoch in range(20):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        print(inputs.shape)
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass ve optimizasyon
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    avg_train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train

    # Validation adımı
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    accuracy = 100 * correct / total

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.2f}%,Train Accuracy: {train_accuracy:.2f}')

print('Training Finished')

torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64, 15, 35, 35])
torch.Size([64

In [ ]:
torch.save(model.state_dict(), 'model_efficient_ip.pth')

In [ ]:
test_dataset = HyperSpectralDataset(
    data=torch.tensor(X_val, dtype=torch.float32),
    labels=torch.tensor(y_val, dtype=torch.long)
    )

X_test_tensor=torch.tensor(X_test)

X_test_tensor.to(device)

X_test_tensor=X_test_tensor.to(device,dtype=torch.float)

a=model(X_test_tensor)

predicted_classes = torch.argmax(a, dim=1)
predicted=predicted_classes.cpu().numpy()

from sklearn.metrics import accuracy_score

print(accuracy_score(predicted,y_test))




0.981184668989547


# Modeli geri yüklemek

In [16]:

####  Resnet #########

model = models.resnet18(pretrained=False)
model.conv1 = nn.Conv2d(15, 64, kernel_size=7, stride=2, padding=3, bias=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 16)
model.load_state_dict(torch.load('model_resnet_ip.pth', map_location=device))
model.to(device)





ResNet(
  (conv1): Conv2d(15, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
 

In [20]:

model = models.densenet121(pretrained=False)
original_conv1 = model.features.conv0
model.features.conv0 = nn.Conv2d(15, original_conv1.out_channels, kernel_size=original_conv1.kernel_size,
                                 stride=original_conv1.stride, padding=original_conv1.padding, bias=False)

num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 16)
model.load_state_dict(torch.load('model_densenet_ip.pth', map_location=device))

model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(15, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [10]:
model = models.googlenet(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 16)
model.load_state_dict(torch.load('model_inception_ip.pth', map_location=device))
model.to(device)
model.eval()

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [ ]:
model = models.efficientnet_b0(pretrained=False)
# Çıkış katmanını değiştir
model.features[0][0] = nn.Conv2d(15, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=16, bias=True)
)
model.load_state_dict(torch.load('model_efficient_ip.pth', map_location=device))
model.to(device)
model.eval()


In [21]:
all_dataset = HyperSpectralDataset(
    data=torch.tensor(X, dtype=torch.float32),
    labels=torch.tensor(y, dtype=torch.long)
    )

all_loader = DataLoader(all_dataset, batch_size=64, shuffle=False)

model.to(device)
model.eval()

# Tahmin işlemi
all_predictions = []
with torch.no_grad():
    for images, _ in all_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.tolist())

In [22]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, cohen_kappa_score

In [23]:
print(accuracy_score(all_predictions,y),
      cohen_kappa_score(all_predictions,y),
f1_score(all_predictions,y,average='macro')
)

0.9620450775685433 0.9568196380259113 0.9213396641722907
